In [2]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from openpyxl import Workbook
from bs4 import BeautifulSoup


# Function to scrape all text from a webpage
def scrape_webpage(url):
    # Set up Selenium WebDriver
    options = Options()
    options.headless = True  # Run in headless mode
    driver = webdriver.Chrome(
        service=ChromeService(ChromeDriverManager().install()), options=options
    )

    try:
        # Open the webpage
        driver.get(url)
        time.sleep(5)  # Wait for the page to load, adjust the time as necessary

        # Initialize a list to store extracted text elements
        text_elements = []

        # Find all text elements
        soup = BeautifulSoup(driver.page_source, "html.parser")
        for element in soup.find_all(text=True):
            if element.parent.name not in [
                "style",
                "script",
                "head",
                "title",
                "meta",
                "[document]",
            ]:
                text_elements.append(element.strip())

        # Get placeholder text from input tags
        for input_tag in soup.find_all("input"):
            placeholder = input_tag.get("placeholder")
            if placeholder:
                text_elements.append(placeholder)

        # Find the span element with class 'select2-selection__placeholder'
        span_tags = soup.find_all("span", class_="select2-selection__placeholder")
        if span_tags:
            for span_tag in span_tags:
                text = span_tag.get_text(strip=True)
                text_elements.append(text)
        else:
            print("No span tags found with class 'select2-selection__placeholder'")

        # Filter out empty and whitespace-only strings
        text_list = [text for text in text_elements if text.strip()]

        return text_list
    finally:
        driver.quit()


# Main function
def main():
    url = "https://css.jne.co.id/login"  # Replace with the URL of the webpage you want to scrape
    output_file = "scraped_data.xlsx"

    # Scrape the webpage to get all text content
    scraped_text = scrape_webpage(url)

    # Write to Excel file
    wb = Workbook()
    ws = wb.active
    ws.title = "Scraped Data"

    # Write each distinct piece of text to a new cell in column A
    for idx, text in enumerate(scraped_text, start=1):
        ws.cell(row=idx, column=1, value=text)

    # Save the workbook
    wb.save(output_file)
    print(f"Scraped data saved to {output_file}")


if __name__ == "__main__":
    main()

C:\Users\mocha\AppData\Local\Temp\ipykernel_24580\1167413270.py:30: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for element in soup.find_all(text=True):


No span tags found with class 'select2-selection__placeholder'
Scraped data saved to scraped_data.xlsx
